# Relative quantification proof of concept

Take a first pass at implementing evaluation of (fractional) relative quantification, more or less following [Joseph's blueprint](https://docs.google.com/document/d/1u18BSWB2776d-EXAoft7Je5KmhC1aGMFZJRpJ1rueNI/edit) with a [few practical pointers I had come up with](https://docs.google.com/document/d/1t30SEHZ6RSFYXG7o1ylWCfgYWVbZCA1jj-SkES1M2rw/edit)

Purpose isn't to necessarily to make something fully flexible/ready to go, but to explore how to implement the blueprint and understand any impacts our filtering criteria/decisions may have.


Broadly, this involves:
- Defining list of genes expressed above a given TPM threshold from RNA-seq data
- Defining a set of terminal exons w/ 2 polyA sites from the GT poly(A) site BEDs, filtering to 2 most expressed sites if a TE has > 2 PAS
- Filtering the predicted sites to those occurring on curated set of terminal exons
- 


Will focus on a single Mayr sample Mayr_CD5B_R3 with outputs from DaPars2 & DaPars for now...

In [4]:
import pyranges as pr
import pandas as pd
import numpy as np
import os
import warnings

# There are some FutureWarnings with installed version of PyRanges and Pandas - For visual purposes just don't print them for now
warnings.filterwarnings('ignore')

# 1. Find genes expressed in RNA-seq above a given threshold


Matt provided a TSV of Salmon quantification outputs from the nf-core RNA-seq workflow (**TODO: insert link**). As this is per-transcript, need to sum to gene-level

Target is a list/set of Ensembl gene IDs that pass a given TPM threshold

In [5]:
tpm_df = pd.read_csv("data/Mayr_salmon.merged.transcript_tpm.tsv", sep="\t")
tpm_df.head(n=10)

,tx,gene_id,Mayr_CD5B_R3,Mayr_CD5B_R4,Mayr_GC_R1,Mayr_GC_R2,Mayr_GC_R3,Mayr_GC_R4,Mayr_M_R2,Mayr_M_R6,Mayr_NB_R1,Mayr_NB_R2,Mayr_NB_R3,Mayr_NB_R4,Mayr_NB_R5,Mayr_NB_R6
0,ENST00000456328.2,ENSG00000223972.5,0.00000,0.044865,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.135770,0.012529,0.000000,0.000000,0.000000,0.00000
1,ENST00000450305.2,ENSG00000223972.5,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
2,ENST00000488147.1,ENSG00000227232.5,2.62799,3.314172,2.877079,2.625817,2.839663,3.452287,2.285098,2.630034,2.627121,2.015704,2.529293,1.699014,2.747853,2.22484
3,ENST00000619216.1,ENSG00000278267.1,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
4,ENST00000473358.1,ENSG00000243485.5,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
5,ENST00000469289.1,ENSG00000243485.5,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.03989
6,ENST00000607096.1,ENSG00000284332.1,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
7,ENST00000417324.1,ENSG00000237613.2,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
8,ENST00000461467.1,ENSG00000237613.2,0.00000,0.000000,0.000000,0.026530,0.000000,0.000000,0.030134,0.000000,0.000000,0.000000,0.026825,0.000000,0.000000,0.00000
9,ENST00000606857.1,ENSG00000268020.3,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.073951,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000


In [6]:
# Summarise individual transcript expression to gene-level TPM
sample_cols = [col for col in tpm_df.columns if col not in ["tx", "gene_id"]]
# sample_cols

gene_tpm = tpm_df.groupby("gene_id")[sample_cols].sum()
gene_tpm

,Mayr_CD5B_R3,Mayr_CD5B_R4,Mayr_GC_R1,Mayr_GC_R2,Mayr_GC_R3,Mayr_GC_R4,Mayr_M_R2,Mayr_M_R6,Mayr_NB_R1,Mayr_NB_R2,Mayr_NB_R3,Mayr_NB_R4,Mayr_NB_R5,Mayr_NB_R6
gene_id,,,,,,,,,,,,,,
ENSG00000000003.15,0.841496,0.313483,0.224971,0.220765,0.022432,0.569883,0.789282,0.378343,0.021219,0.000000,1.269005,0.835454,0.435978,0.007870
ENSG00000000005.6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.053663,0.000000,0.000000
ENSG00000000419.14,31.462058,27.354208,35.251502,29.225256,24.685265,34.762234,28.490978,18.881510,35.288736,26.330158,24.401633,22.140657,19.653655,20.561543
ENSG00000000457.14,13.687977,13.619168,6.316804,5.554757,4.834584,5.143130,9.068360,5.792793,8.083517,0.864191,8.795230,7.967781,6.033806,1.219366
ENSG00000000460.17,6.285515,4.806302,22.109690,18.177691,14.991760,14.880926,4.207996,4.209314,1.514664,0.489746,5.427459,3.280607,3.677558,0.469440
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000288721.1,0.299191,0.590024,0.962898,1.188937,0.839328,1.087980,0.350958,0.304280,0.505675,0.029361,0.246273,0.191482,0.293596,0.159076
ENSG00000288722.1,3.003168,0.794827,4.912289,2.726699,1.875441,7.280854,0.602439,2.423762,1.573895,1.523601,4.048628,0.883793,2.006525,1.103560
ENSG00000288723.1,0.032072,0.000000,0.000000,0.000000,0.000000,0.000000,0.029104,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [7]:
# Create a matrix of Booleans for each column for whether gene passes expression filter
thresh_tpm1 = gene_tpm > 1
thresh_tpm1

,Mayr_CD5B_R3,Mayr_CD5B_R4,Mayr_GC_R1,Mayr_GC_R2,Mayr_GC_R3,Mayr_GC_R4,Mayr_M_R2,Mayr_M_R6,Mayr_NB_R1,Mayr_NB_R2,Mayr_NB_R3,Mayr_NB_R4,Mayr_NB_R5,Mayr_NB_R6
gene_id,,,,,,,,,,,,,,
ENSG00000000003.15,False,False,False,False,False,False,False,False,False,False,True,False,False,False
ENSG00000000005.6,False,False,False,False,False,False,False,False,False,False,False,False,False,False
ENSG00000000419.14,True,True,True,True,True,True,True,True,True,True,True,True,True,True
ENSG00000000457.14,True,True,True,True,True,True,True,True,True,False,True,True,True,True
ENSG00000000460.17,True,True,True,True,True,True,True,True,True,False,True,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000288721.1,False,False,False,True,False,True,False,False,False,False,False,False,False,False
ENSG00000288722.1,True,False,True,True,True,True,False,True,True,True,True,False,True,True
ENSG00000288723.1,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [8]:
# Number of genes passing threshold in each sample
thresh_tpm1.sum(axis=0)

Mayr_CD5B_R3    12718
Mayr_CD5B_R4    12288
Mayr_GC_R1      12662
Mayr_GC_R2      12436
Mayr_GC_R3      11942
Mayr_GC_R4      12673
Mayr_M_R2       12103
Mayr_M_R6       12321
Mayr_NB_R1      11639
Mayr_NB_R2       9477
Mayr_NB_R3      12245
Mayr_NB_R4      11996
Mayr_NB_R5      12070
Mayr_NB_R6       9645
dtype: int64

In [9]:
# Fraction of annotated genes passing threshold in each sample
# NB: could subset to prot-coding & lncRNA genes (as expect their RNAs to be polyadenylated)
# don't think this is that informative as don't expect all annotated genes to be expressed...
thresh_tpm1.sum(axis=0) / len(thresh_tpm1)

Mayr_CD5B_R3    0.209698
Mayr_CD5B_R4    0.202608
Mayr_GC_R1      0.208775
Mayr_GC_R2      0.205049
Mayr_GC_R3      0.196903
Mayr_GC_R4      0.208956
Mayr_M_R2       0.199558
Mayr_M_R6       0.203153
Mayr_NB_R1      0.191908
Mayr_NB_R2      0.156260
Mayr_NB_R3      0.201899
Mayr_NB_R4      0.197794
Mayr_NB_R5      0.199014
Mayr_NB_R6      0.159030
dtype: float64

In [10]:
# Dict of {sample_name: {gene_ids}} that pass the expression filter
genes_thresh_tpm1 = {col: set(thresh_tpm1.index[thresh_tpm1.loc[:, col]]) for col in thresh_tpm1.columns}

# 2 - Find / Decide TEs to select for filtering ground truth

- Merge/'union' overlapping TEs to create the longest possible TE isoform for searching in GT for overlapping sites (most permissive)
    - In cases where TE isoforms have different 3'ss, they will be merged regardless (i.e. not considered as separate events)



In [11]:
%%time
gtf = pr.read_gtf("data/gencode.v38.annotation.gtf")


CPU times: user 1min 14s, sys: 4.43 s, total: 1min 18s
Wall time: 1min 19s


In [12]:
# PyRanges objects are dictionaries of pandas dataframes
# Keys are chrom names & strand, values are dfs storing genomic coords and metadata
gtf.head()

,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,gene_id,gene_type,...,transcript_type,transcript_name,transcript_support_level,tag,havana_transcript,exon_number,exon_id,ont,protein_id,ccdsid
0,chr1,HAVANA,gene,11868,14409,.,+,.,ENSG00000223972.5,transcribed_unprocessed_pseudogene,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,chr1,HAVANA,transcript,11868,14409,.,+,.,ENSG00000223972.5,transcribed_unprocessed_pseudogene,...,processed_transcript,DDX11L1-202,1,basic,OTTHUMT00000362751.1,NaN,NaN,NaN,NaN,NaN
2,chr1,HAVANA,exon,11868,12227,.,+,.,ENSG00000223972.5,transcribed_unprocessed_pseudogene,...,processed_transcript,DDX11L1-202,1,basic,OTTHUMT00000362751.1,1,ENSE00002234944.1,NaN,NaN,NaN
3,chr1,HAVANA,exon,12612,12721,.,+,.,ENSG00000223972.5,transcribed_unprocessed_pseudogene,...,processed_transcript,DDX11L1-202,1,basic,OTTHUMT00000362751.1,2,ENSE00003582793.1,NaN,NaN,NaN
4,chr1,HAVANA,exon,13220,14409,.,+,.,ENSG00000223972.5,transcribed_unprocessed_pseudogene,...,processed_transcript,DDX11L1-202,1,basic,OTTHUMT00000362751.1,3,ENSE00002312635.1,NaN,NaN,NaN
5,chr1,HAVANA,transcript,12009,13670,.,+,.,ENSG00000223972.5,transcribed_unprocessed_pseudogene,...,transcribed_unprocessed_pseudogene,DDX11L1-201,NA,Ensembl_canonical,OTTHUMT00000002844.2,NaN,NaN,PGO:0000019,NaN,NaN
6,chr1,HAVANA,exon,12009,12057,.,+,.,ENSG00000223972.5,transcribed_unprocessed_pseudogene,...,transcribed_unprocessed_pseudogene,DDX11L1-201,NA,Ensembl_canonical,OTTHUMT00000002844.2,1,ENSE00001948541.1,PGO:0000019,NaN,NaN
7,chr1,HAVANA,exon,12178,12227,.,+,.,ENSG00000223972.5,transcribed_unprocessed_pseudogene,...,transcribed_unprocessed_pseudogene,DDX11L1-201,NA,Ensembl_canonical,OTTHUMT00000002844.2,2,ENSE00001671638.2,PGO:0000019,NaN,NaN


In [13]:
gtf.stranded

True

In [35]:
gtf = gtf[["Feature", "gene_id", "transcript_id", "gene_name", "exon_number", "gene_type", "transcript_type", "transcript_support_level"]]

gtf

,Chromosome,Feature,Start,End,Strand,gene_id,transcript_id,gene_name,exon_number,gene_type,transcript_type,transcript_support_level
0,chr1,gene,11868,14409,+,ENSG00000223972.5,NaN,DDX11L1,NaN,transcribed_unprocessed_pseudogene,NaN,NaN
1,chr1,transcript,11868,14409,+,ENSG00000223972.5,ENST00000456328.2,DDX11L1,NaN,transcribed_unprocessed_pseudogene,processed_transcript,1
2,chr1,exon,11868,12227,+,ENSG00000223972.5,ENST00000456328.2,DDX11L1,1,transcribed_unprocessed_pseudogene,processed_transcript,1
3,chr1,exon,12612,12721,+,ENSG00000223972.5,ENST00000456328.2,DDX11L1,2,transcribed_unprocessed_pseudogene,processed_transcript,1
4,chr1,exon,13220,14409,+,ENSG00000223972.5,ENST00000456328.2,DDX11L1,3,transcribed_unprocessed_pseudogene,processed_transcript,1
...,...,...,...,...,...,...,...,...,...,...,...,...
3150419,chrY,exon,57214349,57214397,-,ENSG00000227159.8_PAR_Y,ENST00000507418.6_PAR_Y,DDX11L16,1,unprocessed_pseudogene,unprocessed_pseudogene,NA
3150420,chrY,exon,57213879,57213964,-,ENSG00000227159.8_PAR_Y,ENST00000507418.6_PAR_Y,DDX11L16,2,unprocessed_pseudogene,unprocessed_pseudogene,NA
3150421,chrY,exon,57213525,57213602,-,ENSG00000227159.8_PAR_Y,ENST00000507418.6_PAR_Y,DDX11L16,3,unprocessed_pseudogene,unprocessed_pseudogene,NA
3150422,chrY,exon,57213203,57213357,-,ENSG00000227159.8_PAR_Y,ENST00000507418.6_PAR_Y,DDX11L16,4,unprocessed_pseudogene,unprocessed_pseudogene,NA


In [36]:
# For simplicities sake let's focus on a single sample
# Mayr_CD5B_R3
exons = gtf.subset(lambda df: df["Feature"] == "exon")
# Need as int dtype for some downstream functions to work
exons = exons.assign("exon_number", lambda df: df["exon_number"].astype(float).astype(int))

exons.dtypes


Chromosome                  category
Feature                       object
Start                          int32
End                            int32
Strand                      category
gene_id                       object
transcript_id                 object
gene_name                     object
exon_number                    int64
gene_type                     object
transcript_type               object
transcript_support_level      object
dtype: object

In [37]:
exons_mayr_cd_r3_tpm_filt = exons.subset(lambda df: df["gene_id"].isin(genes_thresh_tpm1["Mayr_CD5B_R3"]))

exons_mayr_cd_r3_tpm_filt

,Chromosome,Feature,Start,End,Strand,gene_id,transcript_id,gene_name,exon_number,gene_type,transcript_type,transcript_support_level
0,chr1,exon,629061,629433,+,ENSG00000225972.1,ENST00000416931.1,MTND1P23,1,unprocessed_pseudogene,unprocessed_pseudogene,NA
1,chr1,exon,629639,630683,+,ENSG00000225630.1,ENST00000457540.1,MTND2P28,1,unprocessed_pseudogene,unprocessed_pseudogene,NA
2,chr1,exon,631073,632616,+,ENSG00000237973.1,ENST00000414273.1,MTCO1P12,1,unprocessed_pseudogene,unprocessed_pseudogene,NA
3,chr1,exon,633695,634376,+,ENSG00000248527.1,ENST00000514057.1,MTATP6P1,1,unprocessed_pseudogene,unprocessed_pseudogene,NA
4,chr1,exon,634375,634922,+,ENSG00000198744.5,ENST00000416718.2,MTCO3P12,1,unprocessed_pseudogene,unprocessed_pseudogene,NA
...,...,...,...,...,...,...,...,...,...,...,...,...
894990,chrY,exon,20506768,20507456,-,ENSG00000229236.3,ENST00000666666.1,TTTY10,4,lncRNA,lncRNA,NaN
894991,chrY,exon,20570221,20570519,-,ENSG00000277438.1,ENST00000622542.1,KDM5DP1,1,processed_pseudogene,processed_pseudogene,NA
894992,chrY,exon,21044598,21044724,-,ENSG00000254488.1,ENST00000527562.1,RP11-65G9.1,1,lncRNA,lncRNA,1
894993,chrY,exon,21042268,21042369,-,ENSG00000254488.1,ENST00000527562.1,RP11-65G9.1,2,lncRNA,lncRNA,1


In [16]:
# Extract terminal exons for each transcript
def get_terminal_exons(gr,
                         feature_col = "Feature",
                         feature_key = "exon",
                         id_col = "transcript_id",
                         region_number_col = "exon_number",
                         filter_single = False
                         ):
    '''
    Return gr of last exons for each transcript
    
    Function makes the following assumptions:
    - region_number_col values define non-overlapping regions (exons) of the same group (transcript_id)
    - region_number_col is strand aware 5'-3' in ascending order
        - i.e. 1st exon (most 5') is numbered 1, last exon of transcript with n exons is labelled n
        - For reference GTFs this is probably a fair assumption
    - ONLY a single unique value (feature_key) is present in feature_col (e.g. only 'exon' present)
    - region_number_col is an int np.dtype col
    - regions are not duplicated within the gr
    
    Notes:
    - use add_region_number if not confident region_number_col matches expectation
        
    '''

    assert region_number_col in gr.columns.tolist()
    assert feature_col in gr.columns.tolist()
    assert id_col in gr.columns.tolist()

    # Make sure region_number is an int dtype (so can be sorted numerically)
    assert gr.dtypes.loc[region_number_col] in [np.dtype('int32'), np.dtype('int64')]
    # Make sure only 'exon' features are in the gr
    assert gr.as_df()[feature_col].drop_duplicates().tolist() == [feature_key], "only {} entries should be present in gr".format(feature_key)


    # Make sure gr is sorted by transcript_id & 'region number' (ascending order so 1..n)
    mod_gr = gr.apply(lambda df: df.sort_values(by=[id_col, region_number_col], ascending=True),
                          nb_cpu=1)


    # Filter out single-exon transcripts
    if filter_single:
        print("Filtering for multi-exon transcripts...")
        print(f"Before: {len(set(mod_gr.as_df()[id_col]))}")

        # Setting to 'False' marks all duplicates as True (so keeps transcript IDs with multiple exons these)
        mod_gr = mod_gr.subset(lambda df: df.duplicated(subset=[id_col], keep=False), nb_cpu=1)

        eprint(f"After: {len(set(mod_gr.as_df()[id_col]))}")

    # Pick last region entry by max region number for each transcript (id_col)
    # keep="last" sets last in ID to 'False' and all others true (negate to keep last only)

    out_gr = mod_gr.subset(lambda df: ~(df.duplicated(subset=[id_col], keep="last")),
                           nb_cpu=1
                           )

    # re-sort by genomic coords not transcript_id
    return out_gr.sort()


def _df_add_region_number(df, id_col, sort_col="Start"):
    '''
    Return a Series of strand-aware region numbers (5'-3' in 1..n)
    Function to be used internally in a pr.assign (mainly by add_region_number)
    '''
    if id_col not in df.columns.tolist():
        raise KeyError(f"id_col - {id_col} - is not present in df for chr/strand pair {','.join([df.Chromosome.iloc[0], df.Strand.iloc[0]])}")

    elif (df.Strand == "+").all():
        # Start position smallest to largest = 5'-3'

        return df.groupby(id_col)[sort_col].rank(method="min", ascending=True)

    elif (df.Strand == "-").all():
        # Start position largest to smallest = 5'-3'

        return df.groupby(id_col)[sort_col].rank(method="min", ascending=False)

    elif df.empty:
        eprint("df is empty - returning empty pd.Series")
        return pd.Series()


def add_region_number(gr,
                      id_col="transcript_id",
                      out_col="exon_number",
                      feature_col="Feature",
                      feature_key="exon",
                      ):
    '''
    Adds column to gr containing a strand aware region number column,
    ordered 5'-3' 1..n by a group of features (e.g. transcript)
    
    Assumes every region/row of feature in group is not duplicated (i.e. given exon only appears once)
    '''

    # Make sure only 'feature_key' rows are in the gr
    assert gr.as_df()[feature_col].drop_duplicates().tolist() == [feature_key], "only {} entries should be present in gr".format(feature_key)

    # Make sure sorted by position first.
    gr = gr.sort()

    # Add in region number column in strand aware manner, so 1 = most 5', n = most 3'

    gr = gr.assign(out_col, lambda df: _df_add_region_number(df, id_col))

    return gr

In [38]:
te_mayr_cd_r3_tpm_filt = get_terminal_exons(exons_mayr_cd_r3_tpm_filt)

te_mayr_cd_r3_tpm_filt

,Chromosome,Feature,Start,End,Strand,gene_id,transcript_id,gene_name,exon_number,gene_type,transcript_type,transcript_support_level
0,chr1,exon,629061,629433,+,ENSG00000225972.1,ENST00000416931.1,MTND1P23,1,unprocessed_pseudogene,unprocessed_pseudogene,NA
1,chr1,exon,629639,630683,+,ENSG00000225630.1,ENST00000457540.1,MTND2P28,1,unprocessed_pseudogene,unprocessed_pseudogene,NA
2,chr1,exon,631073,632616,+,ENSG00000237973.1,ENST00000414273.1,MTCO1P12,1,unprocessed_pseudogene,unprocessed_pseudogene,NA
3,chr1,exon,633695,634376,+,ENSG00000248527.1,ENST00000514057.1,MTATP6P1,1,unprocessed_pseudogene,unprocessed_pseudogene,NA
4,chr1,exon,634375,634922,+,ENSG00000198744.5,ENST00000416718.2,MTCO3P12,1,unprocessed_pseudogene,unprocessed_pseudogene,NA
...,...,...,...,...,...,...,...,...,...,...,...,...
119897,chrY,exon,20503208,20507456,-,ENSG00000229236.3,ENST00000667496.1,TTTY10,2,lncRNA,lncRNA,NaN
119898,chrY,exon,20504475,20507456,-,ENSG00000229236.3,ENST00000659275.1,TTTY10,2,lncRNA,lncRNA,NaN
119899,chrY,exon,20506768,20507456,-,ENSG00000229236.3,ENST00000666666.1,TTTY10,4,lncRNA,lncRNA,NaN
119900,chrY,exon,20570221,20570519,-,ENSG00000277438.1,ENST00000622542.1,KDM5DP1,1,processed_pseudogene,processed_pseudogene,NA


In [39]:
# Merge TEs into non-overlapping, union TEs
m_te_mayr_cd_r3_tpm_filt = te_mayr_cd_r3_tpm_filt.merge(strand=True, by="gene_id")
m_te_mayr_cd_r3_tpm_filt

,Chromosome,Start,End,Strand,gene_id
0,chr1,169804074,169804386,+,ENSG00000000460.17
1,chr1,169807790,169807837,+,ENSG00000000460.17
2,chr1,169821678,169821719,+,ENSG00000000460.17
3,chr1,169852789,169854080,+,ENSG00000000460.17
4,chr1,196672977,196673407,+,ENSG00000000971.16
...,...,...,...,...,...
48897,chrY,20570221,20570519,-,ENSG00000277438.1
48898,chrY,11153857,11154070,-,ENSG00000278212.2
48899,chrY,11155653,11155884,-,ENSG00000278212.2
48900,chrY,11162449,11163137,-,ENSG00000278212.2


In [42]:
%%time
# If want to retain metadata (e.g. transcript IDs) for overlapping TEs, can use functions below
# This approach is a fair bit slower so won't use ds (as just need merged regions), but relevant for Matt's suggestion of a terminal exon ID
# could probably speed up with nb_cpu for multiple processors...
def _collapse_tes(df,
                  cluster_col,
                  distinct_cols
                  ):
    '''
    '''
 
    col_order = df.columns.tolist()
            
    other_cols = [col for col in col_order if col not in distinct_cols]
    # First define how want to collapse metadata columns to single row values
    # 
    # Cols want to combine values as ; separated string (e.g. if expect diff values per row)
    cat_dict = {col: lambda x: x.astype(str).str.cat(sep=";") 
                  for col in df.columns
                    if col not in distinct_cols}

    # Cols want to report a single value (e.g. where expect same values in all rows/not important)
    # Also important where need to maintain original 
    distinct_dict = {col: lambda x: x.first()
                         for col in distinct_cols
                        }
    
    # Define the merged ranges
    cat_dict["Start"] = lambda x: x.min()
    cat_dict["End"] = lambda x: x.max()
    
    grpd = df.groupby(cluster_col, group_keys=False)
    
    grpd_distinct = grpd[distinct_cols].first()
    grpd_other = grpd[other_cols].agg(cat_dict)
    
    # Combine to single row
    clpsd = grpd_distinct.merge(grpd_other, left_index=True, right_index=True)
    
    return clpsd[col_order]


def collapse_tes(gr,
                 cluster_col="Cluster",
                 distinct_cols=["Chromosome", "Feature", "gene_id", "Strand", "gene_name"]):
    
    # Want to keep grouping col to single value
    dist_cols = distinct_cols + [cluster_col]
    
    return gr.apply(lambda df: _collapse_tes(df, cluster_col, dist_cols))

# i = 0 
# for _, df in tmp_grp:
#     while i < 2:
#         print(df)
#         tmp_df = df.copy()
#         cat_dict = {col: lambda x: x.astype(str).str.cat(sep=";") 
#                     for col in tmp_df.columns
#                     if col not in ["Chromosome", "Start", "End", "Feature", "gene_id", "Strand"]}

#         distinct_dict = {col: lambda x: x.head(n=1)
#                          for col in ["Chromosome", "Feature", "gene_id", "Strand"]
#                         }
        
#         agg_dict = {** cat_dict, ** distinct_dict}
        
#         agg_dict["Start"] = lambda x: x.min()
#         agg_dict["End"] = lambda x: x.max()
        
#         print(df.agg(agg_dict))
                
#         i +=1
#     else:
#         break
        
collapse_tes(te_mayr_cd_r3_tpm_filt.cluster(strand=True, by="gene_id"))

CPU times: user 53.7 s, sys: 11.7 ms, total: 53.7 s
Wall time: 53.7 s


,Chromosome,Feature,Start,End,Strand,gene_id,transcript_id,gene_name,exon_number,gene_type,transcript_type,transcript_support_level,Cluster
0,chr1,exon,169804074,169804386,+,ENSG00000000460.17,ENST00000472795.5;ENST00000496973.5,C1orf112,6;6,protein_coding;protein_coding,protein_coding;protein_coding,2;1,1
1,chr1,exon,169807790,169807837,+,ENSG00000000460.17,ENST00000481744.5,C1orf112,7,protein_coding,nonsense_mediated_decay,3,2
2,chr1,exon,169821678,169821719,+,ENSG00000000460.17,ENST00000466580.6,C1orf112,8,protein_coding,nonsense_mediated_decay,4,3
3,chr1,exon,169852789,169854080,+,ENSG00000000460.17,ENST00000413811.3;ENST00000498289.5;ENST000004...,C1orf112,23;29;23;24;25,protein_coding;protein_coding;protein_coding;p...,protein_coding;processed_transcript;nonsense_m...,1;2;2;1;1,4
4,chr1,exon,196672977,196673407,+,ENSG00000000971.16,ENST00000496761.1,CFH,2,protein_coding,processed_transcript,2,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
48897,chrY,exon,20570221,20570519,-,ENSG00000277438.1,ENST00000622542.1,KDM5DP1,1,processed_pseudogene,processed_pseudogene,NA,48898
48898,chrY,exon,11153857,11154070,-,ENSG00000278212.2,ENST00000620700.2,MAFIP,6,transcribed_unprocessed_pseudogene,transcribed_unprocessed_pseudogene,NA,48899
48899,chrY,exon,11155653,11155884,-,ENSG00000278212.2,ENST00000652671.1,MAFIP,3,transcribed_unprocessed_pseudogene,processed_transcript,nan,48900
48900,chrY,exon,11162449,11163137,-,ENSG00000278212.2,ENST00000651211.1,MAFIP,3,transcribed_unprocessed_pseudogene,processed_transcript,nan,48901


In [44]:
# To do/check: if we have unstranded RNA_seq data, consider removing TEs that have any overlap with TEs of diff genes?

(m_te_mayr_cd_r3_tpm_filt.join(m_te_mayr_cd_r3_tpm_filt,
                               how="left",
                               )
 .subset(lambda df: df["gene_id"] != df["gene_id_b"])
 .as_df()
 [["Strand", "Strand_b"]].value_counts()
)

Strand  Strand_b
+       -           799
-       +           799
+       +           332
-       -           292
dtype: int64

**NB: I don't think above is massively important but keeping for completeness**

A very small minority of TEs overlap with TEs of other gene_ids... Most of these overlaps occur on opposite strands, so algos that can profile txpn strand & prediction datasets that are strand-specific should be able to differentiate.

TODO: check overlap of TEs with other exons (i.e. not just terminal exons). Do we want to remove? Maybe not necessary, don't know if tools try to handle this uniformly. GT is just 3'seq (mostly) so should define the PASs. May inflate the number of FNs if a tool removes these kinds of sites.

In [45]:
def assign_id(gr, cols_to_cat=["Chromosome","Start","End","Strand"], sep_char=":", out_col="pas_id"):
    '''
    '''
    
    return gr.assign(out_col,
                     lambda df: df[cols_to_cat[0]].str.cat(df[cols_to_cat[1:]].astype(str),
                                                           sep=sep_char
                                                           )
                    )

m_te_mayr_cd_r3_tpm_filt = assign_id(m_te_mayr_cd_r3_tpm_filt,
                                     cols_to_cat=["Chromosome","Start","End","Strand", "gene_id"],
                                     out_col="te_id")
m_te_mayr_cd_r3_tpm_filt

,Chromosome,Start,End,Strand,gene_id,te_id
0,chr1,169804074,169804386,+,ENSG00000000460.17,chr1:169804074:169804386:+:ENSG00000000460.17
1,chr1,169807790,169807837,+,ENSG00000000460.17,chr1:169807790:169807837:+:ENSG00000000460.17
2,chr1,169821678,169821719,+,ENSG00000000460.17,chr1:169821678:169821719:+:ENSG00000000460.17
3,chr1,169852789,169854080,+,ENSG00000000460.17,chr1:169852789:169854080:+:ENSG00000000460.17
4,chr1,196672977,196673407,+,ENSG00000000971.16,chr1:196672977:196673407:+:ENSG00000000971.16
...,...,...,...,...,...,...
48897,chrY,20570221,20570519,-,ENSG00000277438.1,chrY:20570221:20570519:-:ENSG00000277438.1
48898,chrY,11153857,11154070,-,ENSG00000278212.2,chrY:11153857:11154070:-:ENSG00000278212.2
48899,chrY,11155653,11155884,-,ENSG00000278212.2,chrY:11155653:11155884:-:ENSG00000278212.2
48900,chrY,11162449,11163137,-,ENSG00000278212.2,chrY:11162449:11163137:-:ENSG00000278212.2


Have a set of TEs that pass expression filter, now can filter the GT PAS for those overlapping these TEs


# 3. Select TEs that have APA (>= 2 polyA sites)

Want to also track:
1. Number of PAS that are lost to filtering



In [47]:
gt_pas = pr.read_bed("data/gt/Mayr_CD5B_R3.SRR6795684.3seq.hg38.bed")
gt_pas

,Chromosome,Start,End,Name,Score,Strand
0,1,629238,629239,1:629219-629259:+,13.2345,+
1,1,630540,630541,1:630520-630563:+,58.5904,+
2,1,634373,634374,1:634350-634378:+,87.2652,+
3,1,1014536,1014537,1:1014512-1014544:+,38.4628,+
4,1,1235003,1235004,1:1235003-1235008:+,0.4136,+
...,...,...,...,...,...,...
13338,Y,13323034,13323035,Y:13323029-13323059:-,9.5123,-
13339,Y,19691944,19691945,Y:19691920-19691966:-,11.0288,-
13340,Y,19692471,19692472,Y:19692471-19692501:-,8.4094,-
13341,Y,19705419,19705420,Y:19705410-19705440:-,153.7135,-


In [48]:
gt_pas.Chromosome = "chr" + gt_pas.Chromosome.astype(str)
gt_pas

,Chromosome,Start,End,Name,Score,Strand
0,chr1,629238,629239,1:629219-629259:+,13.2345,+
1,chr1,630540,630541,1:630520-630563:+,58.5904,+
2,chr1,634373,634374,1:634350-634378:+,87.2652,+
3,chr1,1014536,1014537,1:1014512-1014544:+,38.4628,+
4,chr1,1235003,1235004,1:1235003-1235008:+,0.4136,+
...,...,...,...,...,...,...
13338,chrY,13323034,13323035,Y:13323029-13323059:-,9.5123,-
13339,chrY,19691944,19691945,Y:19691920-19691966:-,11.0288,-
13340,chrY,19692471,19692472,Y:19692471-19692501:-,8.4094,-
13341,chrY,19705419,19705420,Y:19705410-19705440:-,153.7135,-


In [53]:
gt_pas = assign_id(gt_pas)
gt_pas

,Chromosome,Start,End,Name,Score,Strand,pas_id
0,chr1,629238,629239,1:629219-629259:+,13.2345,+,chr1:629238:629239:+
1,chr1,630540,630541,1:630520-630563:+,58.5904,+,chr1:630540:630541:+
2,chr1,634373,634374,1:634350-634378:+,87.2652,+,chr1:634373:634374:+
3,chr1,1014536,1014537,1:1014512-1014544:+,38.4628,+,chr1:1014536:1014537:+
4,chr1,1235003,1235004,1:1235003-1235008:+,0.4136,+,chr1:1235003:1235004:+
...,...,...,...,...,...,...,...
13338,chrY,13323034,13323035,Y:13323029-13323059:-,9.5123,-,chrY:13323034:13323035:-
13339,chrY,19691944,19691945,Y:19691920-19691966:-,11.0288,-,chrY:19691944:19691945:-
13340,chrY,19692471,19692472,Y:19692471-19692501:-,8.4094,-,chrY:19692471:19692472:-
13341,chrY,19705419,19705420,Y:19705410-19705440:-,153.7135,-,chrY:19705419:19705420:-


In [20]:
m_te_mayr_cd_r3_tpm_filt.Chromosome.unique().tolist()

['chr1',
 'chr2',
 'chr3',
 'chr4',
 'chr5',
 'chr6',
 'chr7',
 'chr8',
 'chr9',
 'chr10',
 'chr11',
 'chr12',
 'chr13',
 'chr14',
 'chr15',
 'chr16',
 'chr17',
 'chr18',
 'chr19',
 'chr20',
 'chr21',
 'chr22',
 'chrM',
 'chrX',
 'chrY']

In [54]:
n_gt_pas = gt_pas.pas_id.nunique()
n_gt_pas

13343

How many GT PAS do not overlap filtered terminal exons?

In [56]:

gt_pas = pr.PyRanges(gt_pas.as_df(), int64=True)
m_te_mayr_cd_r3_tpm_filt = pr.PyRanges(m_te_mayr_cd_r3_tpm_filt.as_df(), int64=True)

# pr.join often complains if the int dtypes of coordinates are different between PyRanges/not int64
n_gt_pas_no_olap = (gt_pas.join(m_te_mayr_cd_r3_tpm_filt,
             how="left", strandedness="same")
 .subset(lambda df: df["te_id"] == "-1")
 .pas_id.nunique()
)

n_gt_pas_no_olap

5051

In [57]:
n_gt_pas_no_olap / n_gt_pas

0.37855055085063327

~ 38 % do not overlap with filtered TEs...

How many PAS occur on TEs by themselves? i.e. only a single GT PAS for that TE

In [62]:
n_gt_pas_single = (m_te_mayr_cd_r3_tpm_filt.count_overlaps(gt_pas, strandedness="same")
 .subset(lambda df: df["NumberOverlaps"] == 1)
 # Since each TE only has one overlapping pas, can use te_id count as proxy for pas IDs
 .te_id.nunique()
)

n_gt_pas_single

4730

In [63]:
n_gt_pas_single / n_gt_pas

0.35449299258037925

In [71]:
n_gt_pas_min2 = n_gt_pas - n_gt_pas_no_olap - n_gt_pas_single
n_gt_pas_min2

3562

Only 3562 / 13343 PAS are overlapping filtered TEs for our analysis. This number of PAS will shrink as select 2 representative PAS for TEs that have > 2

In [66]:
# TEs vs number of overlapping PAS
# Again perhaps not so surprising to see large propn not having APA
## Isoform not necessarily expressed, TE only has 1 pas etc.
(m_te_mayr_cd_r3_tpm_filt.count_overlaps(gt_pas, strandedness="same")
 .as_df()["NumberOverlaps"].describe(percentiles = [i * 0.01 for i in range(0,100,5)]))

count    48902.00000
mean         0.17077
std          0.55754
min          0.00000
0%           0.00000
5%           0.00000
10%          0.00000
15%          0.00000
20%          0.00000
25%          0.00000
30%          0.00000
35%          0.00000
40%          0.00000
45%          0.00000
50%          0.00000
55%          0.00000
60%          0.00000
65%          0.00000
70%          0.00000
75%          0.00000
80%          0.00000
85%          0.00000
90%          1.00000
95%          1.00000
max         34.00000
Name: NumberOverlaps, dtype: float64

Now subset to valid TEs (at least two PAS)

In [67]:
# TEs have at least 2 overlapping PAS
valid_tes = (m_te_mayr_cd_r3_tpm_filt.count_overlaps(gt_pas, strandedness="same")
             .subset(lambda df: df["NumberOverlaps"] >= 2))

valid_tes

,Chromosome,Start,End,Strand,gene_id,te_id,NumberOverlaps
0,chr1,24468985,24472976,+,ENSG00000001461.17,chr1:24468985:24472976:+:ENSG00000001461.17,2
1,chr1,11840142,11843143,+,ENSG00000011021.23,chr1:11840142:11843143:+:ENSG00000011021.23,2
2,chr1,150158818,150160065,+,ENSG00000023902.14,chr1:150158818:150160065:+:ENSG00000023902.14,4
3,chr1,12508892,12512047,+,ENSG00000048707.15,chr1:12508892:12512047:+:ENSG00000048707.15,2
4,chr1,16395037,16398145,+,ENSG00000055070.17,chr1:16395037:16398145:+:ENSG00000055070.17,2
...,...,...,...,...,...,...,...
1411,chrX,72272041,72272772,-,ENSG00000198034.11,chrX:72272041:72272772:-:ENSG00000198034.11,2
1412,chrX,149477103,149483218,-,ENSG00000241489.8,chrX:149477103:149483218:-:ENSG00000241489.8,2
1413,chrY,19603796,19606274,+,ENSG00000131002.12,chrY:19603796:19606274:+:ENSG00000131002.12,2
1414,chrY,19703864,19706345,-,ENSG00000012817.16,chrY:19703864:19706345:-:ENSG00000012817.16,2


In [68]:
print(f"Number of TEs with at least 2 GT PAS - {valid_tes.te_id.nunique()}")

Number of TEs with at least 2 GT PAS - 1416


In [69]:
# Now subset to PAS overlapping these TEs
gt_pas_tes = gt_pas.join(valid_tes, strandedness="same")

gt_pas_tes

,Chromosome,Start,End,Name,Score,Strand,pas_id,Start_b,End_b,Strand_b,gene_id,te_id,NumberOverlaps
0,chr1,2310137,2310138,1:2310102-2310140:+,118.9729,+,chr1:2310137:2310138:+,2306576,2310213,+,ENSG00000157933.10,chr1:2306576:2310213:+:ENSG00000157933.10,3
1,chr1,2310180,2310181,1:2310180-2310181:+,0.2757,+,chr1:2310180:2310181:+,2306576,2310213,+,ENSG00000157933.10,chr1:2306576:2310213:+:ENSG00000157933.10,3
2,chr1,2310205,2310206,1:2310183-2310220:+,13.5102,+,chr1:2310205:2310206:+,2306576,2310213,+,ENSG00000157933.10,chr1:2306576:2310213:+:ENSG00000157933.10,3
3,chr1,2403215,2403216,1:2403190-2403237:+,31.7077,+,chr1:2403215:2403216:+,2403034,2405442,+,ENSG00000157916.20,chr1:2403034:2405442:+:ENSG00000157916.20,3
4,chr1,2403757,2403758,1:2403743-2403782:+,10.6152,+,chr1:2403757:2403758:+,2403034,2405442,+,ENSG00000157916.20,chr1:2403034:2405442:+:ENSG00000157916.20,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3616,chrY,19605813,19605814,Y:19605788-19605818:+,41.7715,+,chrY:19605813:19605814:+,19603796,19606274,+,ENSG00000131002.12,chrY:19603796:19606274:+:ENSG00000131002.12,2
3617,chrY,19691944,19691945,Y:19691920-19691966:-,11.0288,-,chrY:19691944:19691945:-,19691940,19694606,-,ENSG00000260197.1,chrY:19691940:19694606:-:ENSG00000260197.1,2
3618,chrY,19692471,19692472,Y:19692471-19692501:-,8.4094,-,chrY:19692471:19692472:-,19691940,19694606,-,ENSG00000260197.1,chrY:19691940:19694606:-:ENSG00000260197.1,2
3619,chrY,19705419,19705420,Y:19705410-19705440:-,153.7135,-,chrY:19705419:19705420:-,19703864,19706345,-,ENSG00000012817.16,chrY:19703864:19706345:-:ENSG00000012817.16,2



## 4. Select two representative PAS for TEs with > 2 PASs

If TE has 2 PAS - those are the selected PAS for that TE.
All other TEs - need to select two representative PAS, Joseph's suggestion was to take the two most highly expressed.



In [70]:
# Apparently the rank approach is much faster if we're worried about that...
# https://stackoverflow.com/questions/62157558/filter-for-rows-with-n-largest-values-for-each-group
# Score is the TPM expression of a PAS
# Select two highest expressed PASs for genes 

gt_pas_tes_2rep = (gt_pas_tes.subset(lambda df: df["NumberOverlaps"] > 2)
 .apply(lambda df: df.groupby("te_id").apply(lambda x: x.nlargest(2, "Score")).reset_index(drop=True))
)

gt_pas_tes_2rep

,Chromosome,Start,End,Name,Score,Strand,pas_id,Start_b,End_b,Strand_b,gene_id,te_id,NumberOverlaps
0,chr1,101241516,101241517,1:101241488-101241534:+,102.2919,+,chr1:101241516:101241517:+,101238821,101241518,+,ENSG00000170989.10,chr1:101238821:101241518:+:ENSG00000170989.10,3
1,chr1,101241381,101241382,1:101241358-101241382:+,9.6502,+,chr1:101241381:101241382:+,101238821,101241518,+,ENSG00000170989.10,chr1:101238821:101241518:+:ENSG00000170989.10,3
2,chr1,108700364,108700365,1:108700340-108700378:+,26.6069,+,chr1:108700364:108700365:+,108699161,108702928,+,ENSG00000134186.12,chr1:108699161:108702928:+:ENSG00000134186.12,3
3,chr1,108701192,108701193,1:108701170-108701193:+,13.7860,+,chr1:108701192:108701193:+,108699161,108702928,+,ENSG00000134186.12,chr1:108699161:108702928:+:ENSG00000134186.12,3
4,chr1,110899919,110899920,1:110899894-110899942:+,820.5406,+,chr1:110899919:110899920:+,110899123,110899922,+,ENSG00000143119.14,chr1:110899123:110899922:+:ENSG00000143119.14,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
881,chrX,78961834,78961835,X:78961813-78961835:+,10.2016,+,chrX:78961834:78961835:+,78960507,78963727,+,ENSG00000078589.13,chrX:78960507:78963727:+:ENSG00000078589.13,3
882,chrX,107713326,107713327,X:107713301-107713352:-,19.8518,-,chrX:107713326:107713327:-,107713220,107714749,-,ENSG00000157514.17,chrX:107713220:107714749:-:ENSG00000157514.17,3
883,chrX,107713236,107713237,X:107713236-107713250:-,11.1666,-,chrX:107713236:107713237:-,107713220,107714749,-,ENSG00000157514.17,chrX:107713220:107714749:-:ENSG00000157514.17,3
884,chrX,1386250,1386251,X:1386225-1386271:-,212.5796,-,chrX:1386250:1386251:-,1386151,1386759,-,ENSG00000169100.14,chrX:1386151:1386759:-:ENSG00000169100.14,4


In [72]:
# Combine selected 2 with other PAS on TEs with only 2 PAS to get final list of GT PAS
gt_pas_tes_rep = pr.concat([gt_pas_tes.subset(lambda df: df["NumberOverlaps"] == 2), gt_pas_tes_2rep])
gt_pas_tes_rep

,Chromosome,Start,End,Name,Score,Strand,pas_id,Start_b,End_b,Strand_b,gene_id,te_id,NumberOverlaps
0,chr1,11843045,11843046,1:11843016-11843046:+,13.0967,+,chr1:11843045:11843046:+,11840142,11843143,+,ENSG00000011021.23,chr1:11840142:11843143:+:ENSG00000011021.23,2
1,chr1,11843127,11843128,1:11843102-11843143:+,48.8023,+,chr1:11843127:11843128:+,11840142,11843143,+,ENSG00000011021.23,chr1:11840142:11843143:+:ENSG00000011021.23,2
2,chr1,12013470,12013471,1:12013468-12013476:+,17.2325,+,chr1:12013470:12013471:+,12011495,12013514,+,ENSG00000116688.18,chr1:12011495:12013514:+:ENSG00000116688.18,2
3,chr1,12013501,12013502,1:12013488-12013523:+,20.5411,+,chr1:12013501:12013502:+,12011495,12013514,+,ENSG00000116688.18,chr1:12011495:12013514:+:ENSG00000116688.18,2
4,chr1,12510360,12510361,1:12510335-12510378:+,19.1625,+,chr1:12510360:12510361:+,12508892,12512047,+,ENSG00000048707.15,chr1:12508892:12512047:+:ENSG00000048707.15,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2827,chrY,19605813,19605814,Y:19605788-19605818:+,41.7715,+,chrY:19605813:19605814:+,19603796,19606274,+,ENSG00000131002.12,chrY:19603796:19606274:+:ENSG00000131002.12,2
2828,chrY,19691944,19691945,Y:19691920-19691966:-,11.0288,-,chrY:19691944:19691945:-,19691940,19694606,-,ENSG00000260197.1,chrY:19691940:19694606:-:ENSG00000260197.1,2
2829,chrY,19692471,19692472,Y:19692471-19692501:-,8.4094,-,chrY:19692471:19692472:-,19691940,19694606,-,ENSG00000260197.1,chrY:19691940:19694606:-:ENSG00000260197.1,2
2830,chrY,19705419,19705420,Y:19705410-19705440:-,153.7135,-,chrY:19705419:19705420:-,19703864,19706345,-,ENSG00000012817.16,chrY:19703864:19706345:-:ENSG00000012817.16,2


In [74]:
n_gt_pas_rep = gt_pas_tes_rep.pas_id.nunique()
n_gt_pas_rep

2818

Should be 2 PAS for every TE, but 1416 * 2 != 2818...

In [75]:
# Label as proximal / distal - assign region number 1 (proximal) & 2 (distal)
gt_pas_tes_rep = add_region_number(gt_pas_tes_rep.assign("Feature",
                                                         lambda x: pd.Series(["pas"]*len(x.index))),
                                   id_col="te_id",
                                   out_col="pas_number",
                                   feature_key="pas").drop("Feature")

gt_pas_tes_rep

,Chromosome,Start,End,Name,Score,Strand,pas_id,Start_b,End_b,Strand_b,gene_id,te_id,NumberOverlaps,pas_number
0,chr1,2310137,2310138,1:2310102-2310140:+,118.9729,+,chr1:2310137:2310138:+,2306576,2310213,+,ENSG00000157933.10,chr1:2306576:2310213:+:ENSG00000157933.10,3,1.0
1,chr1,2310205,2310206,1:2310183-2310220:+,13.5102,+,chr1:2310205:2310206:+,2306576,2310213,+,ENSG00000157933.10,chr1:2306576:2310213:+:ENSG00000157933.10,3,2.0
2,chr1,2403215,2403216,1:2403190-2403237:+,31.7077,+,chr1:2403215:2403216:+,2403034,2405442,+,ENSG00000157916.20,chr1:2403034:2405442:+:ENSG00000157916.20,3,1.0
3,chr1,2405434,2405435,1:2405417-2405448:+,13.2345,+,chr1:2405434:2405435:+,2403034,2405442,+,ENSG00000157916.20,chr1:2403034:2405442:+:ENSG00000157916.20,3,2.0
4,chr1,11843045,11843046,1:11843016-11843046:+,13.0967,+,chr1:11843045:11843046:+,11840142,11843143,+,ENSG00000011021.23,chr1:11840142:11843143:+:ENSG00000011021.23,2,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2827,chrY,19605813,19605814,Y:19605788-19605818:+,41.7715,+,chrY:19605813:19605814:+,19603796,19606274,+,ENSG00000131002.12,chrY:19603796:19606274:+:ENSG00000131002.12,2,2.0
2828,chrY,19691944,19691945,Y:19691920-19691966:-,11.0288,-,chrY:19691944:19691945:-,19691940,19694606,-,ENSG00000260197.1,chrY:19691940:19694606:-:ENSG00000260197.1,2,2.0
2829,chrY,19692471,19692472,Y:19692471-19692501:-,8.4094,-,chrY:19692471:19692472:-,19691940,19694606,-,ENSG00000260197.1,chrY:19691940:19694606:-:ENSG00000260197.1,2,1.0
2830,chrY,19705419,19705420,Y:19705410-19705440:-,153.7135,-,chrY:19705419:19705420:-,19703864,19706345,-,ENSG00000012817.16,chrY:19703864:19706345:-:ENSG00000012817.16,2,2.0


In [76]:
# Calculate fractional relative expression for each PAS (vs all PAS on TE)
gt_pas_tes_rep = gt_pas_tes_rep.assign("rel_exp",
                                       lambda df: df.groupby("te_id", group_keys=False)["Score"].apply(lambda x: x / x.sum())
                                       )

gt_pas_tes_rep

,Chromosome,Start,End,Name,Score,Strand,pas_id,Start_b,End_b,Strand_b,gene_id,te_id,NumberOverlaps,pas_number,rel_exp
0,chr1,2310137,2310138,1:2310102-2310140:+,118.9729,+,chr1:2310137:2310138:+,2306576,2310213,+,ENSG00000157933.10,chr1:2306576:2310213:+:ENSG00000157933.10,3,1.0,0.898023
1,chr1,2310205,2310206,1:2310183-2310220:+,13.5102,+,chr1:2310205:2310206:+,2306576,2310213,+,ENSG00000157933.10,chr1:2306576:2310213:+:ENSG00000157933.10,3,2.0,0.101977
2,chr1,2403215,2403216,1:2403190-2403237:+,31.7077,+,chr1:2403215:2403216:+,2403034,2405442,+,ENSG00000157916.20,chr1:2403034:2405442:+:ENSG00000157916.20,3,1.0,0.705522
3,chr1,2405434,2405435,1:2405417-2405448:+,13.2345,+,chr1:2405434:2405435:+,2403034,2405442,+,ENSG00000157916.20,chr1:2403034:2405442:+:ENSG00000157916.20,3,2.0,0.294478
4,chr1,11843045,11843046,1:11843016-11843046:+,13.0967,+,chr1:11843045:11843046:+,11840142,11843143,+,ENSG00000011021.23,chr1:11840142:11843143:+:ENSG00000011021.23,2,1.0,0.211582
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2827,chrY,19605813,19605814,Y:19605788-19605818:+,41.7715,+,chrY:19605813:19605814:+,19603796,19606274,+,ENSG00000131002.12,chrY:19603796:19606274:+:ENSG00000131002.12,2,2.0,0.780928
2828,chrY,19691944,19691945,Y:19691920-19691966:-,11.0288,-,chrY:19691944:19691945:-,19691940,19694606,-,ENSG00000260197.1,chrY:19691940:19694606:-:ENSG00000260197.1,2,2.0,0.567378
2829,chrY,19692471,19692472,Y:19692471-19692501:-,8.4094,-,chrY:19692471:19692472:-,19691940,19694606,-,ENSG00000260197.1,chrY:19691940:19694606:-:ENSG00000260197.1,2,1.0,0.432622
2830,chrY,19705419,19705420,Y:19705410-19705440:-,153.7135,-,chrY:19705419:19705420:-,19703864,19706345,-,ENSG00000012817.16,chrY:19703864:19706345:-:ENSG00000012817.16,2,2.0,0.936188


# Match predictions to filtered ground truth

- Need to assign pred PAS to the correct TE - info not present in prediction BED files

**Really need to tackle problem of multiple predictions matching GT  here (due to tools running tx-by-tx)**

My initial suggestion was to take the two sites that have the 'best match' to ground truth. i.e. Take all PAS assigned to that composite/merged TE, and select the two isoforms that have the best match to the GT, in order of (both match, one of proximal/distal matches, no matches) With way I've named DaPars2 Name field there isn't an out of the box way to know which predictions have come from the same TE frame (would just have to extract the transcript ID). Since DaPars tools calc rel expression tx by tx, there could be a case where the proximal and distal sites match between different transcript models. This could mean that the predicted relative expressions do not sum to 1.


Could probably get around this by deciding on one of prox/distal relative expr to evaluate/compare against. In this case if have multiple predicted pas in a given window, could either take the site with the smallest error/difference to GT relative expression, or take sth like the mean/median of the predicted matched sites rel exp. If did it this way, would maybe be good to try both ways around (i.e. calc metrics relative to distal rel usage and then proximal rel usage).
         

In [77]:
pred_mayr_cd5_dapars2 = pr.read_bed("data/pred/Mayr_CD5B_R3_DaPars2_04.bed")
pred_mayr_cd5_dapars = pr.read_bed("data/pred/Mayr_CD5B_R3_relative_usage_quantification_output.bed")


# Name is not necessarily unique,to be safe (if need it) assign a ID for each PAS
pred_mayr_cd5_dapars2 = assign_id(pred_mayr_cd5_dapars2)
pred_mayr_cd5_dapars = assign_id(pred_mayr_cd5_dapars)

pred_mayr_cd5_dapars2

,Chromosome,Start,End,Name,Score,Strand,pas_id
0,chr1,28453784,28453785,ENST00000438601.2_RP1-308E4.1_proximal,0.08,+,chr1:28453784:28453785:+
1,chr1,51788223,51788224,ENST00000371714.5_OSBPL9_proximal,0.27,+,chr1:51788223:51788224:+
2,chr1,212103089,212103090,ENST00000475419.5_DTL_proximal,0.62,+,chr1:212103089:212103090:+
3,chr1,202439280,202439281,ENST00000417053.2_RP11-175B9.2_proximal,0.04,+,chr1:202439280:202439281:+
4,chr1,24671536,24671537,ENST00000479034.5_SRRM1_proximal,0.35,+,chr1:24671536:24671537:+
...,...,...,...,...,...,...,...
109649,chrY,13234579,13234580,ENST00000684226.1_UTY_distal,0.67,-,chrY:13234579:13234580:-
109650,chrY,2881682,2881683,ENST00000444242.1_HSFY3P_distal,1.00,-,chrY:2881682:2881683:-
109651,chrY,19707181,19707182,ENST00000415360.1_KDM5D_distal,0.26,-,chrY:19707181:19707182:-
109652,chrY,13234580,13234581,ENST00000684326.1_UTY_distal,0.67,-,chrY:13234580:13234581:-


In [78]:
pred_mayr_cd5_dapars

,Chromosome,Start,End,Name,Score,Strand,pas_id
0,chr1,151034007,151034008,ENST00000271620.8|chr1|151033806-151035713|+,0.30,+,chr1:151034007:151034008:+
1,chr1,6092096,6092097,ENST00000656607.1|chr1|6091863-6092187|+,NaN,+,chr1:6092096:6092097:+
2,chr1,156800811,156800812,ENST00000469071.1|chr1|156800374-156800812|+,0.56,+,chr1:156800811:156800812:+
3,chr1,111766684,111766685,ENST00000680983.1|chr1|111765737-111767243|+,0.26,+,chr1:111766684:111766685:+
4,chr1,32652335,32652336,ENST00000465780.1|chr1|32651914-32652506|+,0.00,+,chr1:32652335:32652336:+
...,...,...,...,...,...,...,...
74179,chrY,2883206,2883207,ENST00000652562.1|chrY|2883110-2883407|-,0.00,-,chrY:2883206:2883207:-
74180,chrY,19705682,19705683,ENST00000382806.6|chrY|19705420-19706345|-,0.18,-,chrY:19705682:19705683:-
74181,chrY,2883109,2883110,ENST00000652562.1|chrY|2883110-2883407|-,1.00,-,chrY:2883109:2883110:-
74182,chrY,13323167,13323168,ENST00000329134.9|chrY|13323034-13323368|-,0.00,-,chrY:13323167:13323168:-


As a dirty readout of potential amount of duplication of PAS due to multiple TE isoforms, see how many times identical pas_ids occur

Of course pas could differ very slightly if specific coords of TE isoforms are different & essentially be the same PAS

In [79]:
print("Fraction of predicted pas_id that are duplicated (i.e. identical coordinates)")
print("DaPars")
print(len(pred_mayr_cd5_dapars.pas_id.value_counts().loc[lambda x: x > 1]) / pred_mayr_cd5_dapars.pas_id.nunique())
print("DaPars2")
print(len(pred_mayr_cd5_dapars2.pas_id.value_counts().loc[lambda x: x > 1]) / pred_mayr_cd5_dapars2.pas_id.nunique())

Fraction of predicted pas_id that are duplicated (i.e. identical coordinates)
DaPars
0.09987056774420351
DaPars2
0.1019118652900368


~ 10 % of polyA sites are identical

In [84]:
# From a TE perspective - how many have >2 overlapping prediction sites?
# In the case of DaPars, this could mean overlapping terminal exon isoforms between different transcript IDs
##  DaPars & DaPars2 use a 2-site model, so number of predicted sites can never exceed expected 2
# how many have no overlapping predictions (or )

# Need to convert back to int32 otherwise get following error...
# ValueError: Buffer dtype mismatch, expected 'const int32_t' but got 'long

temp_dapars = pr.PyRanges(valid_tes.as_df(), int64=False).count_overlaps(pr.PyRanges(pred_mayr_cd5_dapars.as_df(), int64=False),
                         strandedness="same",
                         keep_nonoverlapping=True,
                         overlap_col="NumberOverlaps_pred")
temp_dapars2 = pr.PyRanges(valid_tes.as_df(), int64=False).count_overlaps(pr.PyRanges(pred_mayr_cd5_dapars2.as_df(), int64=False),
                         strandedness="same",
                         keep_nonoverlapping=True,
                         overlap_col="NumberOverlaps_pred")


print(temp_dapars.subset(lambda df: df["NumberOverlaps_pred"] < 2).NumberOverlaps_pred.value_counts())
temp_dapars2.subset(lambda df: df["NumberOverlaps_pred"] < 2).NumberOverlaps_pred.value_counts()


0    247
Name: NumberOverlaps_pred, dtype: int64


0    149
Name: NumberOverlaps_pred, dtype: int64

In [85]:
print("Fraction of curated GT terminal exons that have no predicted APA (i.e. < 2 PAS)")
print("DaPars")
print(len(temp_dapars.subset(lambda df: df["NumberOverlaps_pred"] < 2).NumberOverlaps_pred) / temp_dapars.te_id.nunique())
print("DaPars2")
print(len(temp_dapars2.subset(lambda df: df["NumberOverlaps_pred"] < 2).NumberOverlaps_pred) / temp_dapars2.te_id.nunique())

Fraction of curated GT terminal exons that have no predicted APA (i.e. < 2 PAS)
DaPars
0.17443502824858756
DaPars2
0.10522598870056497


Above is Joseph's suggestion for 'FNs'

In [86]:
# Assign predicted PAS to GT TEs
keep_cols = pred_mayr_cd5_dapars.columns.tolist() + ["te_id"]
pred_mayr_cd5_dapars = (pr.PyRanges(pred_mayr_cd5_dapars.as_df(),int64=True).join(pr.PyRanges(valid_tes.as_df(),int64=True),
                                                          how="left",
                                                          strandedness="same", 
                                                          suffix="_te"))[keep_cols]

pred_mayr_cd5_dapars2 = (pr.PyRanges(pred_mayr_cd5_dapars2.as_df(),int64=True).join(pr.PyRanges(valid_tes.as_df(),
                                                                                                int64=True),
                                                          how="left",
                                                          strandedness="same", 
                                                          suffix="_te"))[keep_cols]

pred_mayr_cd5_dapars2

,Chromosome,Start,End,Name,Score,Strand,pas_id,te_id
0,chr1,28496684,28496685,ENST00000493669.2_PHACTR4_proximal,0.00,+,chr1:28496684:28496685:+,chr1:28496533:28500364:+:ENSG00000204138.13
1,chr1,53048234,53048235,ENST00000478274.6_SCP2_proximal,0.00,+,chr1:53048234:53048235:+,chr1:53047857:53051698:+:ENSG00000116171.19
2,chr1,161120697,161120698,ENST00000368009.7_NIT1_proximal,0.48,+,chr1:161120697:161120698:+,chr1:161120498:161121194:+:ENSG00000158793.14
3,chr1,44778726,44778727,ENST00000396651.8_RPS8_proximal,0.11,+,chr1:44778726:44778727:+,chr1:44777999:44778779:+:ENSG00000142937.12
4,chr1,75762850,75762851,ENST00000473018.3_ACADM_proximal,0.48,+,chr1:75762850:75762851:+,chr1:75761121:75763720:+:ENSG00000117054.15
...,...,...,...,...,...,...,...,...
109741,chrY,1431901,1431902,ENST00000462195.6_PAR_Y_ASMTL_distal,0.90,-,chrY:1431901:1431902:-,-1
109742,chrY,13234579,13234580,ENST00000684226.1_UTY_distal,0.67,-,chrY:13234579:13234580:-,-1
109743,chrY,2881682,2881683,ENST00000444242.1_HSFY3P_distal,1.00,-,chrY:2881682:2881683:-,-1
109744,chrY,19707181,19707182,ENST00000415360.1_KDM5D_distal,0.26,-,chrY:19707181:19707182:-,-1


In [87]:
print("Fraction of predicted PAS that do not overlap with curated terminal exons")
print("DaPars")
print(pred_mayr_cd5_dapars.subset(lambda df: df["te_id"] == "-1").pas_id.nunique() / pred_mayr_cd5_dapars.pas_id.nunique())
print("DaPars2")
print(pred_mayr_cd5_dapars2.subset(lambda df: df["te_id"] == "-1").pas_id.nunique() / pred_mayr_cd5_dapars2.pas_id.nunique())

Fraction of predicted PAS that do not overlap with curated terminal exons
DaPars
0.8895191831386523
DaPars2
0.911604729179559


Taking this approach of curating GT data means excluding vast majority of predicted PAS for both DaPars tools. If we considered predictions within a window to be the same PAS, that may reduce the number
